# Car ride-share potential in mid-size U.S. cities from geographic spread

This notebook supports the IBM Data Science Specialization on Coursera, per official report PDF. For all details, see the PDF

## Extract geographic location of mid-size U.S. cities

The 2017 U.S. census estimate for city size can be obtained from https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk . It is faithfully represented in a corresponding Wikipedia page https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308 (retrieved 3 March 2019) from which it can be easily parsed.

In [1]:
import pandas as pd
import numpy as np

Following https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722 (retrieved 24 Feb 2019) using BeautifulSoup to get a parseable representation of the Wikipedia page, then load the table with all cities into `city_table`:

In [2]:
import requests
url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))

<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></span></a></span></small>
</td></tr></tbody></table>


From `city_table` find all cities with an estimated 2017 population between 300,000 and 400,000 and parse out the latitude and longitude into numeric values:
* City name is the second column (remove references in square brackets if present),
* city state is the third column,
* population is the 4th column (remove thousands-separator commas before interpreting as integer),
* lattitude and longitude is contained in the 11th column, but has to be substring-filtered.

In [3]:
import re

l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']
print(cities_df)

(ignoring empty row)
         City name    City state  Population  Latitude  Longitude
0        Arlington         Texas      396394   32.7007   -97.1247
1      New Orleans     Louisiana      393292   30.0534   -89.9345
2          Wichita        Kansas      390591   37.6907   -97.3459
3        Cleveland          Ohio      385525   41.4785   -81.6794
4            Tampa       Florida      385430   27.9701   -82.4797
5      Bakersfield    California      380874   35.3212  -119.0183
6           Aurora      Colorado      366623   39.6880  -104.6897
7          Anaheim    California      352497   33.8555  -117.7601
8         Honolulu        Hawaii      350395   21.3243  -157.8476
9        Santa Ana    California      334136   33.7363  -117.8830
10       Riverside    California      327728   33.9381  -117.3932
11  Corpus Christi         Texas      325605   27.7543   -97.1734
12       Lexington      Kentucky      321959   38.0407   -84.4583
13        Stockton    California      310496   37.9763 

In [4]:
# persist the DataFrame (at least for a little while)
cities_df.to_csv('cities_df.csv')

## Get venues using Foursquare

<!--
CLIENT_ID = 'NBLOR5JJCSM43LTXYWBQYVJ5U3LMNZ2ULCHERZAZVLJTHBYA' # your Foursquare ID
CLIENT_SECRET = 'D1G4RELNK2MGSOZSO1C4DTGPYBAWHIW0MQJTXWSBTGH2JL41' # your Foursquare Secret
-->

In [7]:

print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


In [8]:
# (optional: restore all of the above data from storaget, import all libraries from above)
import pandas as pd
import numpy as np
import requests
import re

cities_df = pd.read_csv('cities_df.csv')

Define a Foursquare query that gets all venues within a default radius of 500 meters around a latitude and longitude. The number of venues returned is capped at 100 by default.

In [9]:
def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request
    results_raw = requests.get(url)
    results = results_raw.json()["response"]['groups'][0]['items']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    
    return(nearby_venues)

### Test number of venues finder

Using the first record in the cities_df DataFrame, test the function defined above.

In [10]:
print(cities_df.head(1))

   Unnamed: 0  City name City state  Population  Latitude  Longitude
0           0  Arlington      Texas      396394   32.7007   -97.1247


In [11]:
venues_df = getVenuesNearLatLong(cities_df.Latitude[0], cities_df.Longitude[0])
city_name = cities_df['City name'][0]
city_state = cities_df['City state'][0]

found 7 venues within 500 meters of 32.7007/-97.1247


In [12]:
print('{}, {} venues_df.shape = {}'.format(city_name, city_state, venues_df.shape))
venues_df

Arlington, Texas venues_df.shape = (7, 4)


,Latitude,Longitude,Venue,Venue Category
0,32.7007,-97.1247,Krispy Kreme Doughnuts,Donut Shop
1,32.7007,-97.1247,Texas Vision Care,Optical Shop
2,32.7007,-97.1247,Kenner's Kolache Bakery,Breakfast Spot
3,32.7007,-97.1247,Cooper St Bakery,Bakery
4,32.7007,-97.1247,El Pollo Regio,Mexican Restaurant
5,32.7007,-97.1247,Avis Car Rental,Rental Car Location
6,32.7007,-97.1247,Metro Flex,Gym


### Search hex grid around a given coordinate

Define a function that takes a latitude and longitude and return the venues at 6 coordinate points around that location, in a hex grid. Each point in the hex grid will be labeled by an integer as shown in the following diagram around the origin (0,0):

```
        ( -1, 1 )      ( 0, 1 )
                \      /
                 \    /
( -1, 0 )  ---  ( 0, 0 )  ---  ( 1, 0 )
                  /  \
                 /    \
        ( 0, -1 )     ( 1, -1 )
```

Given a latitude and longitude, the entire hex grid can therefore be described by a set of tuples `( x, y )`. The function will collect the venues result and append it to a dictionary that is keyed by these `( x, y )` tuples.

In [31]:
import math

def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[]):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            print('getting coordinate {}...'.format(this_hex))
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius, limit)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


Test the hexagonal grid function on the above city, to see how it functions.

In [32]:
# initialize venues_dict with the venues dataframe at the origin
venues_dict = {}
origin_coord = (0,0)
venues_dict[origin_coord] = venues_df
lat_orig = cities_df.Latitude[0]
long_orig = cities_df.Longitude[0]

# call the hex grid exploration function
new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, (0,0) )

getting coordinate (-1, 0)...
found 3 venues within 500 meters of 32.7007/-97.12843637006064
getting coordinate (1, 0)...
found 8 venues within 500 meters of 32.7007/-97.12096362993937
getting coordinate (0, -1)...
found 10 venues within 500 meters of 32.69797706801414/-97.12656818503032
getting coordinate (0, 1)...
found 11 venues within 500 meters of 32.703422931985855/-97.12283181496969
getting coordinate (-1, 1)...
found 2 venues within 500 meters of 32.703422931985855/-97.12656818503032
getting coordinate (1, -1)...
found 10 venues within 500 meters of 32.69797706801414/-97.12283181496969


Output the result:

In [33]:
print('New coordinates probed: {}\n'.format(new_coords))

for key, entry in venues_dict.items():
    print('Venues at {}:\n{}\n\n'.format(key, entry['Venue Category'])) 

New coordinates probed: [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (1, -1)]

Venues at (0, 1):
0                 Donut Shop
1       Caribbean Restaurant
2               Burger Joint
3         Mexican Restaurant
4               Burger Joint
5        Filipino Restaurant
6     Thrift / Vintage Store
7                Pizza Place
8         Mexican Restaurant
9                 Smoke Shop
10        Mexican Restaurant
Name: Venue Category, dtype: object


Venues at (-1, 1):
0          Dessert Shop
1    Mexican Restaurant
Name: Venue Category, dtype: object


Venues at (0, 0):
0             Donut Shop
1           Optical Shop
2         Breakfast Spot
3                 Bakery
4     Mexican Restaurant
5    Rental Car Location
6                    Gym
Name: Venue Category, dtype: object


Venues at (-1, 0):
0           Optical Shop
1         Breakfast Spot
2    Rental Car Location
Name: Venue Category, dtype: object


Venues at (0, -1):
0           Optical Shop
1         Breakfast Spot
2         

Get a graphical representation of where these venues are, using folium

In [34]:
# import libraries required for folium

import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [35]:
def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start, city_name='(city_name)', city_state='(city_state)', radius_zoom=1.0):
    new_map = folium.Map(location=[lat_orig, long_orig], zoom_start=zoom_start)

    # add markers to map
    for coords, venues in venues_dict.items():
        label = '{}, # venues: {}'.format(coords, venues.shape[0])
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [venues['Latitude'][0], venues['Longitude'][0]],
            radius=venues.shape[0] * radius_zoom,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; right: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map

make_map_from_dict(lat_orig, long_orig, venues_dict, 14, city_name, city_state)

### Interpretation of the test

A quick visual inspection confirms the correctness of the algorithm (the coordinates and expected number of venues matches the text printout).

However, it is also obvious that the city is much larger, and many more venue points would have to be calculated this way. The sandbox account into Foursquare is limited to just under 1000 API queries per day, and we have 19 cities in our data set.

Performing 100 query points for each city amounts to 1900 Foursquare API queries. This can be done over the course of two or three days. 100 query points, however, is just a 10x10 grid. Therefore, increase the search size to 1500 meters, and accept that in certain circles we'll exceed the maximum number of venues returned by Foursquare.

Re-testing with the new parameters:

In [36]:
# initialize venues_dict with the venues dataframe at the origin
venues_dict = {}
origin_coord = (0,0)
lat_orig = cities_df.Latitude[0]
long_orig = cities_df.Longitude[0]
venues_dict[origin_coord] = getVenuesNearLatLong(lat_orig, long_orig, radius=1500)

# call the hex grid exploration function
new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, (0,0), radius=1500)

found 56 venues within 1500 meters of 32.7007/-97.1247
getting coordinate (-1, 0)...
found 57 venues within 1500 meters of 32.7007/-97.13590911018191
getting coordinate (1, 0)...
found 68 venues within 1500 meters of 32.7007/-97.1134908898181
getting coordinate (0, -1)...
found 92 venues within 1500 meters of 32.69253120404243/-97.13030455509096
getting coordinate (0, 1)...
found 62 venues within 1500 meters of 32.708868795957564/-97.11909544490905
getting coordinate (-1, 1)...
found 48 venues within 1500 meters of 32.708868795957564/-97.13030455509096
getting coordinate (1, -1)...
found 79 venues within 1500 meters of 32.69253120404243/-97.11909544490905


In [37]:
make_map_from_dict(lat_orig, long_orig, venues_dict, 14, city_name, city_state)

At the risk of hitting the maximum number of venues returned from Foursquare, the radius 1500 meters will be chosen going forward. The number of cities will be capped at 19, and the number of venue points in the algorithm at 100 so that we can gather all required data within 2-3 days from Foursquare.

In a real analysis, we would of course pay for a Foursquare subscription and get many more data points.

### Manually run the next iteration

The algorithm to be built will then look at the coordinate points found so far, pick the one with the highest number of venues, and perform a hex lookup around that coordinate. In the above city, the most venues were returned at coordinate `(0, -1)`. Manually perform an iteration step around that coordinate:

In [38]:
new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, (0,-1), radius=1500, new_coords=new_coords)

getting coordinate (-1, -1)...
found 47 venues within 1500 meters of 32.69253120404243/-97.14151366527287
getting coordinate (0, -2)...
found 100 venues within 1500 meters of 32.684362408084866/-97.13590911018191
getting coordinate (1, -2)...
found 95 venues within 1500 meters of 32.684362408084866/-97.1247


In [39]:
print('total new coordinates not probed yet: {}'.format(new_coords))

total new coordinates not probed yet: [(-1, 0), (1, 0), (0, 1), (-1, 1), (1, -1), (-1, -1), (0, -2), (1, -2)]


Three new coordinate points were queried from Foursquare, because the other three are already known. Resulting map:

In [40]:
make_map_from_dict(lat_orig, long_orig, venues_dict, 14, city_name, city_state)

Just to get an overview, perform a grid query for venues and plot the result.

In [42]:
for x in range(-4, 5):
    for y in range(-4, 5):
        get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, (x,y), radius=1500)

getting coordinate (-4, -4)...
found 72 venues within 1500 meters of 32.66802481616973/-97.19195466109146
getting coordinate (-4, -2)...
found 21 venues within 1500 meters of 32.684362408084866/-97.18074555090955
getting coordinate (-5, -2)...
found 48 venues within 1500 meters of 32.684362408084866/-97.19195466109146
getting coordinate (-3, -2)...
found 8 venues within 1500 meters of 32.684362408084866/-97.16953644072764
getting coordinate (-4, -1)...
found 10 venues within 1500 meters of 32.69253120404243/-97.1751409958186
getting coordinate (-5, -1)...
found 17 venues within 1500 meters of 32.69253120404243/-97.1863501060005
getting coordinate (-3, -1)...
found 10 venues within 1500 meters of 32.69253120404243/-97.16393188563669
getting coordinate (-4, 0)...
found 16 venues within 1500 meters of 32.7007/-97.16953644072764
getting coordinate (-5, 0)...
found 17 venues within 1500 meters of 32.7007/-97.18074555090955
getting coordinate (-3, 0)...
found 34 venues within 1500 meters of 

found 60 venues within 1500 meters of 32.74154397978783/-97.07425900418141
getting coordinate (4, -4)...
found 68 venues within 1500 meters of 32.66802481616973/-97.10228177963619
getting coordinate (4, -5)...
found 17 venues within 1500 meters of 32.65985602021217/-97.10788633472714
getting coordinate (4, -3)...
found 74 venues within 1500 meters of 32.6761936121273/-97.09667722454523
getting coordinate (4, -2)...
found 47 venues within 1500 meters of 32.684362408084866/-97.09107266945428
getting coordinate (4, -1)...
found 11 venues within 1500 meters of 32.69253120404243/-97.08546811436332
getting coordinate (4, 0)...
found 33 venues within 1500 meters of 32.7007/-97.07986355927237
getting coordinate (4, 1)...
found 58 venues within 1500 meters of 32.708868795957564/-97.07425900418141
getting coordinate (4, 2)...
found 57 venues within 1500 meters of 32.71703759191513/-97.06865444909046
getting coordinate (4, 3)...
found 32 venues within 1500 meters of 32.725206387872696/-97.0630498

That's one way of running up your Foursquare quota! Result:

In [43]:
make_map_from_dict(lat_orig, long_orig, venues_dict, 13, city_name, city_state, radius_zoom=0.25)

### Iterative search algorithm

The following iterative search will be performed:
1. Seed the venues_dict dictionary with the venues at the origin,
2. mark the origin as the first (and only) coordinate point not yet explored,
3. loop through exploring the coordinate point with the highest number of venues, until 100 coordinate points have been tested

In [65]:
def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        num_coords_tested += 1
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state

### Test run on the first city

Manually test the algorithm for the first city in cities_df. The earlier fixed 10x10 grid needs to have overlap with this iteratively acquired geographic venues distribution.

In [56]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(cities_df, 0, max_coords_tested=100)

found 56 venues within 1500 meters of 32.7007/-97.1247
getting coordinate (-1, 0)...
found 57 venues within 1500 meters of 32.7007/-97.13590911018191
getting coordinate (1, 0)...
found 68 venues within 1500 meters of 32.7007/-97.1134908898181
getting coordinate (0, -1)...
found 92 venues within 1500 meters of 32.69253120404243/-97.13030455509096
getting coordinate (0, 1)...
found 62 venues within 1500 meters of 32.708868795957564/-97.11909544490905
getting coordinate (-1, 1)...
found 48 venues within 1500 meters of 32.708868795957564/-97.13030455509096
getting coordinate (1, -1)...
found 79 venues within 1500 meters of 32.69253120404243/-97.11909544490905
getting coordinate (-1, -1)...
found 47 venues within 1500 meters of 32.69253120404243/-97.14151366527287
getting coordinate (0, -2)...
found 100 venues within 1500 meters of 32.684362408084866/-97.13590911018191
getting coordinate (1, -2)...
found 95 venues within 1500 meters of 32.684362408084866/-97.1247
getting coordinate (-1, -2)

found 100 venues within 1500 meters of 32.76605036766053/-97.07986355927237
getting coordinate (-1, 8)...
found 100 venues within 1500 meters of 32.76605036766053/-97.09107266945428
getting coordinate (-2, 7)...
found 70 venues within 1500 meters of 32.75788157170296/-97.10788633472714
getting coordinate (-2, 8)...
found 98 venues within 1500 meters of 32.76605036766053/-97.10228177963619
getting coordinate (2, 7)...
found 78 venues within 1500 meters of 32.75788157170296/-97.0630498939995
getting coordinate (1, 8)...
found 100 venues within 1500 meters of 32.76605036766053/-97.06865444909046
getting coordinate (2, 6)...
found 100 venues within 1500 meters of 32.749712775745394/-97.06865444909046
getting coordinate (0, 9)...
found 52 venues within 1500 meters of 32.7742191636181/-97.07425900418141
getting coordinate (-1, 9)...
found 63 venues within 1500 meters of 32.7742191636181/-97.08546811436332
getting coordinate (-2, 9)...
found 69 venues within 1500 meters of 32.7742191636181/-9

found 32 venues within 1500 meters of 32.63534963233946/-97.1247
getting coordinate (2, 11)...
found 12 venues within 1500 meters of 32.79055675553323/-97.04063167363569
getting coordinate (1, 12)...
found 20 venues within 1500 meters of 32.7987255514908/-97.04623622872664
getting coordinate (0, 12)...
found 24 venues within 1500 meters of 32.7987255514908/-97.05744533890855
getting coordinate (8, -1)...
found 12 venues within 1500 meters of 32.69253120404243/-97.04063167363569
getting coordinate (7, -2)...
found 62 venues within 1500 meters of 32.684362408084866/-97.05744533890855
getting coordinate (8, -2)...
found 90 venues within 1500 meters of 32.684362408084866/-97.04623622872664
getting coordinate (9, -2)...
found 49 venues within 1500 meters of 32.684362408084866/-97.03502711854473
getting coordinate (8, -3)...
found 88 venues within 1500 meters of 32.6761936121273/-97.0518407838176
getting coordinate (9, -3)...
found 92 venues within 1500 meters of 32.6761936121273/-97.0406316

In [61]:
make_map_from_dict(lat_orig, long_orig, venues_dict, 12, city_name, city_state, radius_zoom=0.2)

In [66]:
# test the second city in the list
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(cities_df, 1, max_coords_tested=100)

found 4 venues within 1500 meters of 30.0534/-89.9345
getting coordinate (-1, 0)...
found 8 venues within 1500 meters of 30.0534/-89.94539759660488
getting coordinate (1, 0)...
found 5 venues within 1500 meters of 30.0534/-89.92360240339512
getting coordinate (0, -1)...
found 8 venues within 1500 meters of 30.045231204042434/-89.93994879830244
getting coordinate (0, 1)...
found 3 venues within 1500 meters of 30.061568795957566/-89.92905120169756
getting coordinate (-1, 1)...
found 5 venues within 1500 meters of 30.061568795957566/-89.93994879830244
getting coordinate (1, -1)...
found 5 venues within 1500 meters of 30.045231204042434/-89.92905120169756
getting coordinate (-2, 0)...
found 24 venues within 1500 meters of 30.0534/-89.95629519320975
getting coordinate (-1, -1)...
found 29 venues within 1500 meters of 30.045231204042434/-89.95084639490732
getting coordinate (-2, 1)...
found 6 venues within 1500 meters of 30.061568795957566/-89.95084639490732
getting coordinate (-2, -1)...
fo

found 100 venues within 1500 meters of 29.963543244466766/-90.08161755416589
getting coordinate (-7, -10)...
found 100 venues within 1500 meters of 29.971712040424336/-90.06527115925857
getting coordinate (-7, -11)...
found 100 venues within 1500 meters of 29.963543244466766/-90.07071995756101
getting coordinate (-12, -9)...
found 71 venues within 1500 meters of 29.979880836381902/-90.11431034398053
getting coordinate (-11, -10)...
found 100 venues within 1500 meters of 29.971712040424336/-90.1088615456781
getting coordinate (-12, -8)...
found 40 venues within 1500 meters of 29.988049632339468/-90.1088615456781
getting coordinate (-10, -11)...
found 100 venues within 1500 meters of 29.963543244466766/-90.10341274737566
getting coordinate (-8, -12)...
found 100 venues within 1500 meters of 29.9553744485092/-90.08706635246833
getting coordinate (-7, -12)...
found 100 venues within 1500 meters of 29.9553744485092/-90.07616875586345
getting coordinate (-6, -10)...
found 100 venues within 1

found 83 venues within 1500 meters of 29.91453046872137/-90.05982236095613
getting coordinate (-2, -17)...
found 67 venues within 1500 meters of 29.91453046872137/-90.04892476435126
getting coordinate (-3, -18)...
found 92 venues within 1500 meters of 29.9063616727638/-90.06527115925857
getting coordinate (-2, -18)...
found 100 venues within 1500 meters of 29.9063616727638/-90.0543735626537
getting coordinate (-1, -18)...
found 96 venues within 1500 meters of 29.9063616727638/-90.0434759660488
getting coordinate (-2, -19)...
found 100 venues within 1500 meters of 29.898192876806235/-90.05982236095613
getting coordinate (-1, -19)...
found 89 venues within 1500 meters of 29.898192876806235/-90.04892476435126
getting coordinate (-3, -19)...
found 54 venues within 1500 meters of 29.898192876806235/-90.07071995756101
getting coordinate (-2, -20)...
found 58 venues within 1500 meters of 29.89002408084867/-90.06527115925857
getting coordinate (-1, -20)...
found 82 venues within 1500 meters of

In [68]:
make_map_from_dict(lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=0.2)

Looks like it's working ok. You can see clusters of venues and how they're spread out.

### Determine aggregates describing the geographical distribution

In order to cluster cities by shape of their venues, we need to calculate aggregate variables:
* Determine the center of the venues by averaging each coordinate point, weighed by the number of venues there.
* Determine the mean distance of all venues from that center.
* Determine the standard deviation of the distance distribution, as well as skewedness and kurtosis ("peaky-ness").
Once we have these values, all details on the geographic venues distribution will be discarded.

(We will sum up the number of indicator venues as compared to the total number of venues as well, but that's not geographic information.)